## Imports

In [ ]:
# General imports 
import pandas as pd
import numpy as np
import os
import re
import plotly.express as px
from tqdm import tqdm
import tempfile
import csv
import requests as r
from Bio import SeqIO
from io import StringIO
import matplotlib.pyplot as plt
import seaborn as sns
import scipy


# Import structuremap functions
import structuremap.utils
structuremap.utils.set_logger()
from structuremap.processing import download_alphafold_cif, download_alphafold_pae, format_alphafold_data, annotate_accessibility, get_smooth_score, annotate_proteins_with_idr_pattern, get_extended_flexible_pattern, get_proximity_pvals, perform_enrichment_analysis, perform_enrichment_analysis_per_protein, evaluate_ptm_colocalization, extract_motifs_in_proteome
from structuremap.plotting import plot_enrichment, plot_ptm_colocalization

## Set Parameters of Analysis

In [ ]:
analysis_threshold = 20 # number of amino acids either side to analyze

modifications = ["649.3660", "655.3735"] # which modifications we are looking for, as regex strings
heavy_modification = "655.3735" 
light_modification = "649.3660"

## Load Dataset - All Peptides

In [ ]:
# path for csv output data
datasets_path_str = "../datasets/"
datasets_path = os.path.abspath(datasets_path_str)
print(datasets_path)

In [ ]:
data_loc = os.path.join(datasets_path, "combined_modified_peptide_label_quant.tsv")
peptides = pd.read_csv(data_loc, delimiter="\t")
peptides

In [ ]:
ratio_df = peptides.filter(like='Log2 Ratio HL', axis=1)
ratio_df = ratio_df.fillna(999.999)
ratio_df

In [ ]:
num_hyperreactive_instances = ratio_df[ ratio_df < 1.0 ].count(axis=1)
num_hyperreactive_instances.value_counts()

In [ ]:
peptides["prelim_hyperreactive"] = num_hyperreactive_instances >= 3
peptides

In [ ]:
peptides["prelim_hyperreactive"].value_counts()

In [ ]:
# helper function to get full amino acid sequence for a protein
def get_full_protein_seq(cID):
    baseUrl="http://www.uniprot.org/uniprot/"
    currentUrl=baseUrl+cID+".fasta"
    response = r.post(currentUrl)
    cData=''.join(response.text)
    
    Seq=StringIO(cData)
    pSeq=list(SeqIO.parse(Seq,"fasta"))

    return str(pSeq[0].seq)

In [ ]:
# test - get a single amino acid sequence - TODO: FIX THIS FOR THIS PARTICULAR NOTEBOOK
#first_protein_ID = peptides["Protein ID"].iloc[0]
#test_sequence = get_full_protein_seq(first_protein_ID)
#print(test_sequence[575:587])
#print(peptides["Peptide Sequence"].iloc[0])

In [ ]:
unique_uniprot_IDs = peptides["Protein ID"].unique()
unique_uniprot_IDs, len(unique_uniprot_IDs), len(peptides["Protein ID"])

In [ ]:
#unique_IDs_to_sequence_df = pd.DataFrame({"Protein ID": unique_uniprot_IDs})
#unique_IDs_to_sequence_df

In [ ]:
# get whole amino acid sequences for methionine peptides
# SLOW - ONLY DO THIS ONCE - CONVERT TO CSV FILE, THEN RE-LOAD FROM THAT FILE

#tqdm.pandas()
#unique_IDs_to_sequence_df["Complete Sequence"] = unique_IDs_to_sequence_df["Protein ID"].progress_apply(get_full_protein_seq)
#unique_IDs_to_sequence_df

In [ ]:
#peptides_completed_sequence = peptides.merge(unique_IDs_to_sequence_df, how="left", on="Protein ID")
#peptides_completed_sequence

In [ ]:
#peptides_completed_sequence.to_csv(os.path.join(datasets_path, "hyperreactivity_model_completed_sequence.csv"))

In [ ]:
path = os.path.join(datasets_path, "hyperreactivity_model_completed_sequence.csv")
peptides_completed_sequence = pd.read_csv(path)
peptides_completed_sequence.set_index("Unnamed: 0", inplace=True)
peptides_completed_sequence.index.name = None
peptides_completed_sequence

In [ ]:
# create regex pattern to identify desired modifications
def create_modifications_pattern(modifications):

    split_mod = modifications[0].split(".")
    whole = split_mod[0]
    mantissa = split_mod[1]
    pattern = r"M\[{}\.{}\]".format(whole, mantissa)

    for i in range(1, len(modifications)):
        split_mod = modifications[i].split(".")
        whole = split_mod[0]
        mantissa = split_mod[1]
        pattern += r"|M\[{}\.{}\]".format(whole, mantissa)
    
    return pattern

modifications_pattern = create_modifications_pattern(modifications)
print(modifications_pattern)

In [ ]:
peptides_completed_sequence["Sequence Location"] = pd.Series([a.find(b) for a, b in zip(peptides_completed_sequence["Complete Sequence"], peptides_completed_sequence["Peptide Sequence"])])
peptides_completed_sequence

In [ ]:
peptides_completed_sequence[peptides_completed_sequence["Sequence Location"] == -1]

In [ ]:
# other sequences within the same proteins are found -> so, drop rows 313, 648, and 847 (peptides that weren't found in their protein)

peptides_completed_sequence[peptides_completed_sequence["Protein ID"].isin(["P60660", "Q9Y2K9", "P08727"])]

In [ ]:
peptides_completed_sequence = peptides_completed_sequence.drop([313, 648, 847])
peptides_completed_sequence

In [ ]:
temp = [A[B:B+C] for A, B, C in zip(peptides_completed_sequence["Complete Sequence"], peptides_completed_sequence["Sequence Location"], peptides_completed_sequence["Peptide Length"])]
(temp == peptides_completed_sequence["Peptide Sequence"]).value_counts()

In [ ]:
# create regex pattern to identify desired modifications
left_prefix_pattern = "(.*)(" + modifications_pattern + ")"
print(left_prefix_pattern)

In [ ]:
# extract left prefix of modified methionine (for indexing purposes)
IUPACCodes = "ACDEFGHIKLMNPQRSTVWY"
filtering = lambda string: ''.join([char for char in string if char in IUPACCodes])

peptides_completed_sequence["Left Prefix"] = peptides_completed_sequence["Heavy Modified Peptide"].str.extract(left_prefix_pattern)[0]
peptides_completed_sequence["Left Prefix"] = peptides_completed_sequence["Left Prefix"].fillna("")
peptides_completed_sequence["Left Prefix"] = peptides_completed_sequence["Left Prefix"].map(filtering)
peptides_completed_sequence["Left Prefix Length"] = peptides_completed_sequence["Left Prefix"].str.len()

peptides_completed_sequence

In [ ]:
peptides_completed_sequence["Methionine Location"] = peptides_completed_sequence["Sequence Location"] + peptides_completed_sequence["Left Prefix Length"]
peptides_completed_sequence

In [ ]:
# Compute left/right analysis sequences based on threshold
peptides_completed_sequence[f"Left {analysis_threshold}"] = [A[B-1-analysis_threshold:B-1]  if (B - 1 - analysis_threshold >= 0) else A[0:B-1] for A, B in zip(peptides_completed_sequence["Complete Sequence"], peptides_completed_sequence["Methionine Location"])]
peptides_completed_sequence[f"Right {analysis_threshold}"] = [A[B+1:B+1+analysis_threshold] for A, B in zip(peptides_completed_sequence["Complete Sequence"], peptides_completed_sequence["Methionine Location"])]
peptides_completed_sequence

In [ ]:
temp = pd.Series([A[B] for A, B in zip(peptides_completed_sequence["Complete Sequence"], peptides_completed_sequence["Methionine Location"])])
temp[temp != "M"]

In [ ]:
peptides_completed_sequence.loc[953]

In [ ]:
#peptides_completed_sequence.to_csv(os.path.join(datasets_path, "hyperreactivity_model_completed_sequence_with_thresholds.csv"))

In [ ]:
path = os.path.join(datasets_path, "hyperreactivity_model_completed_sequence_with_thresholds.csv")
peptides_completed_sequence = pd.read_csv(path)
peptides_completed_sequence.set_index("Unnamed: 0", inplace=True)
peptides_completed_sequence.index.name = None
peptides_completed_sequence

# Download Alphafold Data - Labeled Methionines

In [ ]:
# path for alphafold protein data
alphafold_path_str = "../alphafold_data/"
alphafold_path = os.path.abspath(alphafold_path_str)

cif_dir = os.path.join(alphafold_path, "cif")
pae_dir = os.path.join(alphafold_path, "pae")

print(alphafold_path)
print(cif_dir)
print(pae_dir)

In [ ]:
# set uniprot IDs to use
uniprotIDs = peptides_completed_sequence["Protein ID"].values
uniprotIDs

In [ ]:
# download cif data for proteins
# SLOW THE FIRST TIME
valid_proteins_cif, invalid_proteins_cif, existing_proteins_cif = download_alphafold_cif(
    proteins=uniprotIDs,
    out_folder=cif_dir
)

In [ ]:
# download pae data for proteins
# SLOW THE FIRST TIME
valid_proteins_pae, invalid_proteins_pae, existing_proteins_pae = download_alphafold_pae(
    proteins=uniprotIDs,
    out_folder=pae_dir, 
)

## Construct Alphafold Dataframe (Calculate Accessibilities) - Labeled Methionines

In [ ]:
# format alphafold data into dataframe
alphafold_annotation = format_alphafold_data(
    directory=cif_dir, 
    protein_ids=uniprotIDs)
alphafold_annotation

In [ ]:
# calculate full sphere exposure -> radius = 24
full_sphere_exposure_24 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=24, 
    max_angle=180, 
    error_dir=pae_dir)
full_sphere_exposure_24

In [ ]:
alphafold_accessibility = alphafold_annotation.merge(
    full_sphere_exposure_24, how='left', on=['protein_id','AA','position'])
alphafold_accessibility

In [ ]:
# calculate full sphere exposure -> angle = 70, radius = 12
part_sphere_exposure = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=12, 
    max_angle=70, 
    error_dir=pae_dir)
part_sphere_exposure

In [ ]:
alphafold_accessibility = alphafold_accessibility.merge(
    part_sphere_exposure, how='left', on=['protein_id','AA','position'])
alphafold_accessibility

In [ ]:
# calculate full sphere exposure -> radius = 6
full_sphere_exposure_6 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=6, 
    max_angle=180, 
    error_dir=pae_dir)
full_sphere_exposure_6

In [ ]:
alphafold_accessibility = alphafold_accessibility.merge(
    full_sphere_exposure_6, how='left', on=['protein_id','AA','position'])
alphafold_accessibility

In [ ]:
# calculate full sphere exposure -> radius = 12
full_sphere_exposure_12 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=12, 
    max_angle=180, 
    error_dir=pae_dir)
full_sphere_exposure_12

In [ ]:
alphafold_accessibility = alphafold_accessibility.merge(
    full_sphere_exposure_12, how='left', on=['protein_id','AA','position'])
alphafold_accessibility

In [ ]:
# calculate full sphere exposure -> radius = 18
full_sphere_exposure_18 = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=18, 
    max_angle=180, 
    error_dir=pae_dir)
full_sphere_exposure_18

In [ ]:
alphafold_accessibility = alphafold_accessibility.merge(
    full_sphere_exposure_18, how='left', on=['protein_id','AA','position'])
alphafold_accessibility

In [ ]:
alphafold_accessibility_smooth = get_smooth_score(
    alphafold_accessibility, 
    np.array(['nAA_6_180_pae', 'nAA_12_180_pae', 'nAA_18_180_pae', 'nAA_24_180_pae']), 
    [10])
alphafold_accessibility_smooth

In [ ]:
alphafold_accessibility_smooth['IDR'] = np.where(
    alphafold_accessibility_smooth['nAA_24_180_pae_smooth10']<=34.27, 1, 0)
alphafold_accessibility_smooth

# Merge Dataframes into Full Dataset (Includes Alphafold) - Labeled Methionines

In [ ]:
alphafold_accessibility_smooth["position"] -= 1 # zero-index the positions to match initial dataframe

peptides_with_alphafold = peptides_completed_sequence.merge(
    alphafold_accessibility_smooth, 
    how="left", 
    left_on=["Protein ID", "Methionine Location"], 
    right_on=["protein_id", "position"]
)
peptides_with_alphafold

In [ ]:
#peptides_with_alphafold.to_csv(os.path.join(datasets_path, "RvsS_peptides_with_alphafold.csv"))

In [ ]:
path = os.path.join(datasets_path, "RvsS_peptides_with_alphafold.csv")
peptides_with_alphafold = pd.read_csv(path)
peptides_with_alphafold.set_index("Unnamed: 0", inplace=True)
peptides_with_alphafold.index.name = None
peptides_with_alphafold

# Load Dataset (MitoCarta3.0) - Full Mitochondrial Proteome

In [ ]:
data_loc = os.path.join(datasets_path, "Mouse.MitoCarta3.0.xls")
mitocarta3_0 = pd.read_excel(data_loc, sheet_name="A Mouse MitoCarta3.0")
mitocarta3_0

In [ ]:
# calculate number of proteins in the mitochondrial proteome
is_mitochondrial = (mitocarta3_0["HPA_Main_Location_2020 (Reliability)"].str.contains("mitoch", case=False))
(is_mitochondrial == True).value_counts(dropna=False)

In [ ]:
# ensure protein split was done correctly (correct mitoch... string matching)
pd.set_option('display.max_rows', None)
display(mitocarta3_0[is_mitochondrial == True]["HPA_Main_Location_2020 (Reliability)"].value_counts())
pd.reset_option('display.max_rows')

In [ ]:
# filter MitoCarta3.0 dataset to only include mitochondrial proteins
mitocarta3_0_mitochondrial = mitocarta3_0[is_mitochondrial == True]
mitocarta3_0_mitochondrial

In [ ]:
mitocarta3_0_mitochondrial["UniProt"].isna().value_counts()

In [ ]:
# drop rows with NaN UniProt IDs (just one)
mitocarta3_0_mitochondrial.dropna(subset=["UniProt"], inplace=True)
mitocarta3_0_mitochondrial

In [ ]:
# get whole amino acid sequences for mitochondrial proteome
# SLOW - ONLY DO THIS ONCE - CONVERT TO CSV FILE, THEN RE-LOAD FROM THAT FILE

#tqdm.pandas()
#mitocarta3_0_mitochondrial_completed_sequence = mitocarta3_0_mitochondrial.copy()
#mitocarta3_0_mitochondrial_completed_sequence["Complete Sequence"] = mitocarta3_0_mitochondrial_completed_sequence["UniProt"].progress_apply(get_full_protein_seq)
#mitocarta3_0_mitochondrial_completed_sequence

In [ ]:
#mitocarta3_0_mitochondrial_completed_sequence.to_csv(os.path.join(datasets_path, "RvsS_full_mitochondrial_completed_sequence.csv"))

In [ ]:
path = os.path.join(datasets_path, "RvsS_full_mitochondrial_completed_sequence.csv")
mitocarta3_0_mitochondrial_completed_sequence = pd.read_csv(path)
mitocarta3_0_mitochondrial_completed_sequence.set_index("Unnamed: 0", inplace=True)
mitocarta3_0_mitochondrial_completed_sequence.index.name = None
mitocarta3_0_mitochondrial_completed_sequence

In [ ]:
# NOTE: sequence length from database doesn't exactly match up with length of sequence as determined by UniProtID - weird
(mitocarta3_0_mitochondrial_completed_sequence["ProteinLength"] - mitocarta3_0_mitochondrial_completed_sequence["Complete Sequence"].str.len()).value_counts()

# Download Alphafold Data - Full Mitochondrial Proteome

In [ ]:
# path for alphafold protein data
alphafold_path_str = "../alphafold_data/"
alphafold_path = os.path.abspath(alphafold_path_str)

cif_dir = os.path.join(alphafold_path, "cif")
pae_dir = os.path.join(alphafold_path, "pae")

print(alphafold_path)
print(cif_dir)
print(pae_dir)

In [ ]:
# NOTE: these IDs are invalid from anAalphafold perspective - they are the secondary UniProtIDs, which was fine for querying UniProt, but not Alphafold
# so, manually impute these IDs with their primary ones
# invalid_proteins_cif -> ['F7C846', 'Q9JLT4', 'Q3UW66', 'Q91XR9']

replace_dict = {'F7C846': 'Q8R5C0', 'Q3UW66': 'Q99J99', 'Q91XR9': 'O70325'} 
# Q9JLT4 is weird - seems to be primary ID, but not in Alphafold
# Q91XR9 / O70325 is weird - O70325 exists in UniProt, but not Alphafold

mitocarta3_0_mitochondrial_completed_sequence["UniProt-Primary"] = mitocarta3_0_mitochondrial_completed_sequence["UniProt"].replace(replace_dict)
mitocarta3_0_mitochondrial_completed_sequence = mitocarta3_0_mitochondrial_completed_sequence.drop(mitocarta3_0_mitochondrial_completed_sequence[mitocarta3_0_mitochondrial_completed_sequence["UniProt"] == "Q9JLT4"].index)
mitocarta3_0_mitochondrial_completed_sequence = mitocarta3_0_mitochondrial_completed_sequence.drop(mitocarta3_0_mitochondrial_completed_sequence[mitocarta3_0_mitochondrial_completed_sequence["UniProt-Primary"] == "O70325"].index)

In [ ]:
# set uniprot IDs to use
uniprotIDs_fullproteome = mitocarta3_0_mitochondrial_completed_sequence["UniProt-Primary"].values
uniprotIDs_fullproteome

In [ ]:
# download cif data for proteins
# SLOW THE FIRST TIME
valid_proteins_cif, invalid_proteins_cif, existing_proteins_cif = download_alphafold_cif(
    proteins=uniprotIDs_fullproteome,
    out_folder=cif_dir
)

In [ ]:
# download pae data for proteins
# SLOW THE FIRST TIME
valid_proteins_pae, invalid_proteins_pae, existing_proteins_pae = download_alphafold_pae(
    proteins=uniprotIDs_fullproteome,
    out_folder=pae_dir, 
)

# Construct Alphafold Dataframe (Calculate Accessibilities) - Full Mitochondrial Proteome

In [ ]:
# format alphafold data into dataframe
alphafold_annotation_full = format_alphafold_data(
    directory=cif_dir, 
    protein_ids=uniprotIDs_fullproteome)
alphafold_annotation_full

In [ ]:
# calculate full sphere exposure -> radius = 24
full_sphere_exposure_24 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=24, 
    max_angle=180, 
    error_dir=pae_dir)
full_sphere_exposure_24

In [ ]:
alphafold_accessibility_full = alphafold_annotation_full.merge(
    full_sphere_exposure_24, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full

In [ ]:
# calculate full sphere exposure -> angle = 70, radius = 12
part_sphere_exposure = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=12, 
    max_angle=70, 
    error_dir=pae_dir)
part_sphere_exposure

In [ ]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    part_sphere_exposure, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full

In [ ]:
# calculate full sphere exposure -> radius = 6
full_sphere_exposure_6 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=6, 
    max_angle=180, 
    error_dir=pae_dir)
full_sphere_exposure_6

In [ ]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    full_sphere_exposure_6, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full

In [ ]:
# calculate full sphere exposure -> radius = 12
full_sphere_exposure_12 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=12, 
    max_angle=180, 
    error_dir=pae_dir)
full_sphere_exposure_12

In [ ]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    full_sphere_exposure_12, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full

In [ ]:
# calculate full sphere exposure -> radius = 18
full_sphere_exposure_18 = annotate_accessibility(
    df=alphafold_annotation_full, 
    max_dist=18, 
    max_angle=180, 
    error_dir=pae_dir)
full_sphere_exposure_18

In [ ]:
alphafold_accessibility_full = alphafold_accessibility_full.merge(
    full_sphere_exposure_18, how='left', on=['protein_id','AA','position'])
alphafold_accessibility_full

In [ ]:
alphafold_accessibility_full_smooth = get_smooth_score(
    alphafold_accessibility_full, 
    np.array(['nAA_6_180_pae', 'nAA_12_180_pae', 'nAA_18_180_pae', 'nAA_24_180_pae']), 
    [10])
alphafold_accessibility_full_smooth

In [ ]:
alphafold_accessibility_full_smooth['IDR'] = np.where(
    alphafold_accessibility_full_smooth['nAA_24_180_pae_smooth10']<=34.27, 1, 0)
alphafold_accessibility_full_smooth

# ????? Merge Dataframes into Full Dataset (Includes Alphafold) - Full Mitochondrial Proteome

In [ ]:
# filter out table to only include methionines
mitocarta3_0_methionines = alphafold_accessibility_full_smooth[alphafold_accessibility_full_smooth["AA"] == "M"]
mitocarta3_0_methionines

In [ ]:
mitocarta3_0_methionines["position"] = mitocarta3_0_methionines["position"] - 1 # zero-index the positions to match initial dataframe

mitocarta3_0_methionines_with_alphafold = mitocarta3_0_methionines.merge(
    mitocarta3_0_mitochondrial_completed_sequence[["UniProt", "UniProt-Primary", "Complete Sequence"]], 
    how="left", 
    left_on="protein_id", 
    right_on="UniProt-Primary"
)
mitocarta3_0_methionines_with_alphafold

In [ ]:
#mitocarta3_0_methionines_with_alphafold.to_csv(os.path.join(datasets_path, "RvsS_full_mitochondrial_with_alphafold.csv"))

In [ ]:
path = os.path.join(datasets_path, "RvsS_full_mitochondrial_with_alphafold.csv")
mitocarta3_0_methionines_with_alphafold = pd.read_csv(path)
mitocarta3_0_methionines_with_alphafold.set_index("Unnamed: 0", inplace=True)
mitocarta3_0_methionines_with_alphafold.index.name = None
mitocarta3_0_methionines_with_alphafold